In [1]:
!pip install -U ragatouille

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.7/86.7 kB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 67.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 53.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 31.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 95.5 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 326.4/326.4 kB 26.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 70.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 63.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.3/278.3 kB 23.5 MB/s eta 0

In [2]:
!kaggle datasets download -d cryptexcode/mpst-movie-plot-synopses-with-tags -p /kaggle/working/

Dataset URL: https://www.kaggle.com/datasets/cryptexcode/mpst-movie-plot-synopses-with-tags
License(s): CC-BY-SA-4.0
 56%|█████████████████████▏                | 16.0M/28.8M [00:00<00:00, 71.5MB/s]
100%|███████████████████████████████████████| 28.8M/28.8M [00:00<00:00, 107MB/s]


In [3]:
from ragatouille import RAGPretrainedModel

RAG = RAGPretrainedModel.from_pretrained("colbert-ir/colbertv2.0")

artifact.metadata:   0%|          | 0.00/1.63k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/405 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/colbert/utils/amp.py:12: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler()


# Méthode "Wikipedia Movie Plots":




In [10]:
import zipfile
import pandas as pd


with zipfile.ZipFile('/kaggle/working/mpst-movie-plot-synopses-with-tags.zip', 'r') as zip_ref:
    zip_ref.extractall('/kaggle/working/')

df = pd.read_csv("/kaggle/working/mpst_full_data.csv")

collection = df['plot_synopsis'][:100].tolist()
titles = df['title'][:100].tolist()


In [11]:
collection = [
    f"Title: {title}\nPlot: {synopsis}"
    for title, synopsis in zip(titles, collection)
]

In [12]:
print(len(df))

14828


# Créer embeddings + Index Faiss qui sauvegarde les embedding

In [ ]:
!pip uninstall --y faiss-cpu & pip install faiss-gpu

Found existing installation: faiss-cpu 1.9.0.post1
Uninstalling faiss-cpu-1.9.0.post1:
  Successfully uninstalled faiss-cpu-1.9.0.post1


In [13]:
RAG.index(
    collection=collection,
    index_name="movie_plots_index",
    max_document_length=300,
    split_documents=False,
    bsize = 32,
)

New index_name received! Updating current index_name (movie_plots_index) to movie_plots_index
---- WARNING! You are using PLAID with an experimental replacement for FAISS for greater compatibility ----
This is a behaviour change from RAGatouille 0.8.0 onwards.
This works fine for most users and smallish datasets, but can be considerably slower than FAISS and could cause worse results in some situations.
If you're confident with FAISS working on your machine, pass use_faiss=True to revert to the FAISS-using behaviour.
--------------------


[Jan 11, 16:21:06] #> Note: Output directory .ragatouille/colbert/indexes/movie_plots_index already exists


[Jan 11, 16:21:06] #> Will delete 1 files already at .ragatouille/colbert/indexes/movie_plots_index in 20 seconds...
#> Starting...
#> Starting...
#> Joined...
#> Joined...
Done indexing!


'.ragatouille/colbert/indexes/movie_plots_index'

In [32]:
!mv /kaggle/working/.ragatouille /kaggle/working/movie_plots_index

mv: cannot stat '/kaggle/working/.ragatouille': No such file or directory


In [ ]:
RAG.index_name = '/kaggle/working/movie_plots_index'

In [38]:
RAG = RAGPretrainedModel.from_index("/kaggle/working/movie_plots_index/colbert/indexes/movie_plots_index")

In [39]:
results = RAG.search(query="A movie about space exploration and human survival", k=3)

Loading searcher for index movie_plots_index for the first time... This may take a few seconds
[Jan 11, 16:35:32] #> Loading codec...
[Jan 11, 16:35:32] Loading decompress_residuals_cpp extension (set COLBERT_LOAD_TORCH_EXTENSION_VERBOSE=True for more info)...


/usr/local/lib/python3.10/dist-packages/colbert/indexing/codecs/residual.py:141: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  centroids = torch.load(centroids_path, map_loc

[Jan 11, 16:35:32] Loading packbits_cpp extension (set COLBERT_LOAD_TORCH_EXTENSION_VERBOSE=True for more info)...


/usr/local/lib/python3.10/dist-packages/torch/utils/cpp_extension.py:1965: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'].
  warnings.warn(


[Jan 11, 16:35:32] #> Loading IVF...
[Jan 11, 16:35:32] #> Loading doclens...


...

[Jan 11, 16:35:32] #> Loading codes and residuals...



  0%|          | 0/2 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/colbert/indexing/codecs/residual_embeddings.py:86: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.


Searcher loaded!



#> QueryTokenizer.tensorize(batch_text[0], batch_background[0], bsize) ==
#> Input: . A movie about space exploration and human survival, 		 True, 		 None
#> Output IDs: torch.Size([32]), tensor([ 101,    1, 1037, 3185, 2055, 2686, 8993, 1998, 2529, 7691,  102,  103,
         103,  103,  103,  103,  103,  103,  103,  103,  103,  103,  103,  103,
         103,  103,  103,  103,  103,  103,  103,  103], device='cuda:0')
#> Output Mask: torch.Size([32]), tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')



/usr/local/lib/python3.10/dist-packages/colbert/utils/amp.py:15: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  return torch.cuda.amp.autocast() if self.activated else NullContextManager()


In [40]:
for i in range(3):
    print(f"Rank = {results[i]['score']}")
    print(f"Title = {titles[results[i]['passage_id']]}")
    print(f"{results[i]['content']}\n")

Rank = 14.578125
Title = Night of the Lepus
Title: Night of the Lepus
Plot: The movie opens showing a news report about how overpopulation is not just a human problem, but exists in the animal kingdom as well, using the rabbit population explosion in Australia in the 1950s as an example. There was a similar plague of rabbits getting out of hand in the American Southwest, especially in Arizona, where the rabbits were devouring crops, destroying property, and out-competing livestock for a finite supply of food and water.Moving to the action in Arizona, our first view is of rancher Cole Hillman, riding his horse. The horse steps into a rabbit hole and injures itself, and Cole must shoot it and walk home. He instructs one of his ranch hands to take the truck to go get his saddle, then makes a phone call to the university in Ajo, hoping to find a cure for the rabbit infestation. He meets Dr. Elgin Clark, but his first suggestion is for the same person who killed all the coyotes that were ke

# Use as LangChain Retriever

In [ ]:
retriever = RAG.as_langchain_retriever(k=3)

In [ ]:
resultats = retriever.invoke("Movie that talk about animals")

/usr/local/lib/python3.10/dist-packages/colbert/utils/amp.py:15: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  return torch.cuda.amp.autocast() if self.activated else NullContextManager()


In [ ]:
for doc in resultats:
    print("Content:", doc.page_content)  # Contenu principal du document
    print("Metadata:", doc.metadata)

Content: The film begins with animal trainer Fred Mason (Milburn Stone) returning from his latest safari with a horde of animals for his employer John Whipple (Lloyd Corrigan), owner of the Whipple Circus. Among them is Cheela (Ray Corrigan), a gorilla with remarkably human characteristics. Mason relates that she is the most affectionate jungle animal he has ever encountered.
Mason’s fiancée Beth Colman (Evelyn Ankers) is present at the dock for his return. She tells him of the recent health problems encountered by her sister Dorothy (Martha MacVicar). In a flashback sequence, Beth tells of taking her sibling to see Dr. Sigmund Walters (John Carradine), an endocrinologist of some standing. Dorothy is staying at Walters’ Crestview Sanatorium for treatment.
Fred and Beth arrive at the winter quarters, and Dr. Walters pays a visit. He is extremely interested in Cheela, and inquires about purchasing her. Whipple tells him that she is not for sale. Upon returning to his lab, Walters finds t